In [91]:
import pandas as pd 
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split

In [92]:
#!pip install turicreate

In [93]:
import sys
sys.path.append("..")

In [94]:
# /Users/snehabhatt/Documents/DataScience/Datascience_WindowsBackup/DataScience/RecommenderSystem/ItemBasedRecommenderSystem/data

In [95]:
import os as os
os.chdir("/Users/snehabhatt/Documents/DataScience/Datascience_WindowsBackup")
os.listdir()
customers = pd.read_csv('/Users/snehabhatt/Documents/DataScience/Datascience_WindowsBackup/DataScience/RecommenderSystem/ItemBasedRecommenderSystem/data/recommend_1.csv') 
transactions = pd.read_csv('/Users/snehabhatt/Documents/DataScience/Datascience_WindowsBackup/DataScience/RecommenderSystem/ItemBasedRecommenderSystem/data/trx_data.csv')

In [96]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [97]:
transactions.shape
transactions.head()

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [98]:
def pir2(df, c):
    colc = df[c].str.split('|')
    clst = colc.values.tolist()
    lens = [len(l) for l in clst]
    j = df.columns.get_loc(c)
    v = df.values
    n, m = v.shape
    r = np.arange(n).repeat(lens)
    print('c='+str(c))
    print("j="+str(j))
    print("v="+str(v))
    print("n="+str(n))
    print("m="+str(m))
    print("r="+str(r))
    return pd.DataFrame(
        np.column_stack([v[r, 0:j], np.concatenate(clst), v[r, j+1:]]),
        columns=df.columns
    )

In [99]:
transactionsNew=pir2(transactions,'products')

c=products
j=1
v=[[0 '20']
 [1 '2|2|23|68|68|111|29|86|107|152']
 [2 '111|107|29|11|11|11|33|23']
 ...
 [6119 '24']
 [8171 '167']
 [4095 '150']]
n=62483
m=2
r=[    0     1     1 ... 62480 62481 62482]


In [100]:
transactionsNew.head()

,customerId,products
0,0,20
1,1,2
2,1,2
3,1,23
4,1,68


In [101]:
data = pd.melt(transactionsNew.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

In [102]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [103]:
data_dummy.head()

,customerId,productId,purchase_count,purchase_dummy
0,0,1,2,1
1,0,13,1,1
2,0,136,2,1
3,0,157,1,1
4,0,19,3,1


In [104]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [105]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [106]:
df_matrix_norm

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# create a table for input to the modeling  
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [108]:
d.head()

productId,customerId,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
scaled_purchase_freq,,,,,,,,,,,,,,,,,,,,,
0,0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
data_norm.head()

,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [110]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [111]:
data.head()

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,136,2
3,0,157,1
4,0,19,3


In [112]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [113]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [114]:
# constant variables to define field names include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [115]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [116]:
## Popularity Model as Baseline
#The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
#Training data is used for model selection

In [117]:
## using purchase count..

name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23303 users and 300 items.

Data prepared in: 0.148204s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 56443

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    132    | 3.1333333333333333 |  1   |
|    1553    |     34    | 3.0609756097560976 |  2   |
|    1553    |     37    | 3.0155038759689923 |  3   |
|    1553    |    248    |        3.0         |  4   |
|    1553    |     0     | 2.998080614203455  |  5   |
|    1553    |     3     | 2.8155136268343814 |  6   |
|    1553    |    110    | 2.7471264367816093 |  7   |
|    1553    |     27    | 2.6906474820143886 |  8   |
|    1553    |     32    |       2.655        |  9   |
|    1553    |     10    | 2.6005747126436782 |  10  |
|   20400    |    132    | 3.1333333333333333 |  1   |
|   20400    |     34    | 3.0609756097560976 |  2   |
|   20400    |     37    | 3.0155038759689923 |  3   |
|   20400    |    248    |        3.0         |  4   |
|   20400    |     0     | 2.998080614203455  |  5   |
|   20400 

In [118]:
## using purchase dummy....

name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23348 users and 300 items.

Data prepared in: 0.142685s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 79402.9

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     2     |  1.0  |  1   |
|    1553    |    140    |  1.0  |  2   |
|    1553    |     49    |  1.0  |  3   |
|    1553    |    176    |  1.0  |  4   |
|    1553    |    200    |  1.0  |  5   |
|    1553    |    101    |  1.0  |  6   |
|    1553    |    254    |  1.0  |  7   |
|    1553    |    185    |  1.0  |  8   |
|    1553    |     94    |  1.0  |  9   |
|    1553    |     48    |  1.0  |  10  |
|   20400    |     2     |  1.0  |  1   |
|   20400    |    140    |  1.0  |  2   |
|   20400    |     49    |  1.0  |  3   |
|   20400    |    176    |  1.0  |  4   |
|   20400    |    200    |  1.0  |  5   |
|   20400    |    101    |  1.0  |  6   |
|   20400    |    254    |  1.0  |  7   |
|   20400    |    185    |  1.0  |  8   |
|   20400    |     94    |  1.0  |  9   |
|   20400    |     48    |  1.0  |  10  |
|   19750    |     2     |  1.0  |

In [119]:
#using scale purchase count...
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23424 users and 300 items.

Data prepared in: 0.124945s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 80064.1

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7785714285714286 |  1   |
|    1553    |    247    | 0.34745762711864414 |  2   |
|    1553    |    230    | 0.32608695652173864 |  3   |
|    1553    |    248    |  0.2528409090909091 |  4   |
|    1553    |    125    | 0.25079365079365046 |  5   |
|    1553    |    204    | 0.23999999999999982 |  6   |
|    1553    |    294    | 0.23359999999999964 |  7   |
|    1553    |     83    | 0.22598870056497183 |  8   |
|    1553    |    276    | 0.22407407407407406 |  9   |
|    1553    |    155    | 0.22105263157894736 |  10  |
|   20400    |    226    |  0.7785714285714286 |  1   |
|   20400    |    247    | 0.34745762711864414 |  2   |
|   20400    |    230    | 0.32608695652173864 |  3   |
|   20400    |    248    |  0.2528409090909091 |  4   |
|   20400    |    125    | 0.25079365079365046 |

In [120]:
## Collaborative Filtering Model

name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 106868 observations with 23303 users and 300 items.

Data prepared in: 0.123453s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.077209s

recommendations finished on 1000/1000 queries. users per second: 52623.3

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.08404332399368286  |  1   |
|    1553    |     1     | 0.07409733533859253  |  2   |
|    1553    |     5     |  0.0647609035174052  |  3   |
|    1553    |    269    | 0.05799933274586996  |  4   |
|    1553    |     41    | 0.056705594062805176 |  5   |
|    1553    |     61    | 0.05666017532348633  |  6   |
|    1553    |     14    | 0.05518043041229248  |  7   |
|    1553    |     33    | 0.05109453201293945  |  8   |
|    1553    |     76    |  0.0501136581103007  |  9   |
|    1553    |    162    | 0.04933476448059082  |  10  |
|   20400    |     1     | 0.04299294948577881  |  1   |
|   20400    |     13    | 0.04098778963088989  |  2   |
|   20400    |     31    | 0.04076641798019409  |  3   |
|   20400    |    160    | 0.03584146499633789  |  4   |
|   20400    |     77    | 0.03

In [121]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23348 users and 300 items.

Data prepared in: 0.127495s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.093362s

recommendations finished on 1000/1000 queries. users per second: 39022.9

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     2     |  0.0  |  1   |
|    1553    |    140    |  0.0  |  2   |
|    1553    |     49    |  0.0  |  3   |
|    1553    |    176    |  0.0  |  4   |
|    1553    |    200    |  0.0  |  5   |
|    1553    |    101    |  0.0  |  6   |
|    1553    |    254    |  0.0  |  7   |
|    1553    |    185    |  0.0  |  8   |
|    1553    |     94    |  0.0  |  9   |
|    1553    |     48    |  0.0  |  10  |
|   20400    |     2     |  0.0  |  1   |
|   20400    |    140    |  0.0  |  2   |
|   20400    |     49    |  0.0  |  3   |
|   20400    |    176    |  0.0  |  4   |
|   20400    |    200    |  0.0  |  5   |
|   20400    |    101    |  0.0  |  6   |
|   20400    |    254    |  0.0  |  7   |
|   20400    |    185    |  0.0  |  8   |
|   20400    |     94    |  0.0  |  9   |
|   20400    |     48    |  0.0  |  10  |
|   19750    |     2     |  0.0  |

## model evaluation..
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [ ]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)